In [31]:
###Load libraries
import awscli
import boto3
import selenium
import pandas as pd
import time
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import boto3

####SCRAPE THE WEBSITE######
###call the webdriver
s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]') #identifies the location of the EIN element
inputElement.send_keys('0') #sends the "0" as the search value for EIN 
inputElement1 = browser.find_element(By.XPATH,'//*[@id="header"]/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]').click() #instatiates the click of the search
sleep(4) #allow for the page to load by adding a sleep element
#identify the table to scrape
table = browser.find_element(By.CSS_SELECTOR,'table.Bordered')
sleep(1)
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table


pages = 7

for i in range(1,pages+1): 
      page = browser.find_element(By.XPATH,f'/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/span[2]/a[{i}]')
      for row in table.find_elements(By.CSS_SELECTOR,'tr'):
            cols = df.append([cell.text for cell in row.find_elements(By.CSS_SELECTOR,'td')])


#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,None,None,None,None,None,None
1,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926,NFP,PHILADELPHIA,PA
2,"""Incredibly Blessed"" Inc",49-54-61,842071758,NFP,STATEN ISLAND,NY
3,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670,NFP,ROCHESTER,NY
4,"""Studio 5404"" Inc.",44-39-58,463180470,NFP,MASSAPAQUA,NY
...,...,...,...,...,...,...
107,04/11 10:17 PM test,47-13-95,206256427,NFP,ALBANY,NY
108,1/20/21 Action Fund,46-99-13,832210730,NFP,SAN FRANCISCO,CA
109,"10/40 Connections, Inc.",45-70-15,621825230,NFP,HIXSON,TN
110,"1000 Feet Project, Inc",45-00-14,473820859,NFP,NEW YORK,NY


In [36]:
# remove the first blank row
df = df.iloc[1:,:]

In [37]:
df

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
1,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926.0,NFP,PHILADELPHIA,PA
2,"""Incredibly Blessed"" Inc",49-54-61,842071758.0,NFP,STATEN ISLAND,NY
3,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670.0,NFP,ROCHESTER,NY
4,"""Studio 5404"" Inc.",44-39-58,463180470.0,NFP,MASSAPAQUA,NY
5,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128.0,NFP,HUDSON,NY
...,...,...,...,...,...,...
107,04/11 10:17 PM test,47-13-95,206256427.0,NFP,ALBANY,NY
108,1/20/21 Action Fund,46-99-13,832210730.0,NFP,SAN FRANCISCO,CA
109,"10/40 Connections, Inc.",45-70-15,621825230.0,NFP,HIXSON,TN
110,"1000 Feet Project, Inc",45-00-14,473820859.0,NFP,NEW YORK,NY


In [39]:
# AWS credentials
aws_access_key_id = 'xxx'
aws_secret_access_key = 'xxx'

# Initialize Boto3 client for S3
s3_client = boto3.client('s3', 
                         aws_access_key_id=aws_access_key_id, 
                         aws_secret_access_key=aws_secret_access_key)


# Bucket name (replace 'your-unique-bucket-name' with your desired bucket name)
bucket_name = 'm10group1'

# CSV file content
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False)
csv_content = csv_buffer.getvalue()
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp

# Upload CSV file to S3
try:
    s3_client.put_object(Bucket=bucket_name, Key=f'data_{datetime}.csv', Body=csv_content)
    print("CSV file uploaded successfully to S3.")
except Exception as e:
    print("Error uploading CSV file to S3:", e)

CSV file uploaded successfully to S3.


### S3 bucket path
s3://m10group1/data.csv

In [40]:
response = s3_client.get_object(Bucket=bucket_name, Key=f'data_{datetime}.csv')
csv_content = response['Body'].read().decode('utf-8')
df = pd.read_csv(StringIO(csv_content))
df

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,"""Forever Captain Poodaman"" The Ahmad Butler Fo...",48-07-16,843800926.0,NFP,PHILADELPHIA,PA
1,"""Incredibly Blessed"" Inc",49-54-61,842071758.0,NFP,STATEN ISLAND,NY
2,"""R"" S.U.C.C.E.S.S. Foundation Inc.",49-06-59,874012670.0,NFP,ROCHESTER,NY
3,"""Studio 5404"" Inc.",44-39-58,463180470.0,NFP,MASSAPAQUA,NY
4,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128.0,NFP,HUDSON,NY
...,...,...,...,...,...,...
106,04/11 10:17 PM test,47-13-95,206256427.0,NFP,ALBANY,NY
107,1/20/21 Action Fund,46-99-13,832210730.0,NFP,SAN FRANCISCO,CA
108,"10/40 Connections, Inc.",45-70-15,621825230.0,NFP,HIXSON,TN
109,"1000 Feet Project, Inc",45-00-14,473820859.0,NFP,NEW YORK,NY


In [41]:
df.to_csv(f'data_{datetime}.csv')